In [357]:
# Imports 
import string
import pandas as pd
import numpy as np
import time
from scipy.stats.stats import pearsonr 
from sklearn.model_selection import train_test_split

In [285]:
# load the data, keep only selected columns
# Read the file using super fast Pandas.read_csv
def load_required_data(path, required_columns, start_index, end_index):
    dataframe = pd.read_json(path, lines=True)
    dataframe = dataframe[required_columns][start_index:end_index]
    return dataframe, dataframe.values

In [286]:
dataframe, values = load_required_data('../reviews_Apps_for_Android_5.json', ["asin", "reviewerID", "overall"], 0, 20000)

In [375]:
train_df = dataframe.sample(frac=0.7,random_state=200)
test_df = dataframe.drop(train.index)

In [391]:
np.count_nonzero(ratings_matrix, axis=0)

array([  17,   62,   96,   95,    7,    7,   63,    6,  244,   86,   53,
         76,   24,   13,    5,   11,   17,   76,  380,  571,    6,   17,
         19,   17,   33,  122,   18,   36,  355,   14,  119,   22,   39,
         14,   38,   16,  112,  172,  484,    7,  355,    9,   31,   14,
          5,   12,    7,   22,  348,    7,   98,    7,   17,  211,   10,
         85,   12,    5,   18,   67,   64,    5,    5,   49,    5,    5,
         24,  582,   19,    7,    9,  328,   10,   77,    7,   13,    5,
         55,   95,   89,   11,  110,  206,   65,    9,  283,   86,    5,
        327,   44,   44,   69,  144,   30,  482,    8,  189,   42,   96,
         58,   47,  122,  563,   99,    5,  219,    5,   18,    5,   67,
         16,    5,  899,   36,    9,  905,    5,  210, 1066,    5,    6,
         55,   14,    5,  105,    6,   29,    7,   23,   24,   11,    6,
         43,   18,   33,   32,   36,  246,   26,    7,   35,   26,    5,
          8,    5,   72,   15,   54,  720,   98,   

In [236]:
dataframe[:10]

,asin,reviewerID,overall
0,B004A9SDD8,A1N4O8VOJZTDVB,3
1,B004A9SDD8,A2HQWU6HUKIEC7,5
2,B004A9SDD8,A1SXASF6GYG96I,5
3,B004A9SDD8,A2B54P9ZDYH167,5
4,B004A9SDD8,AFOFZDTX5UC6D,5
5,B004A9SDD8,A331GYAT4ESYI3,5
6,B004A9SDD8,A2YEHF8T823TDC,5
7,B004A9SDD8,A3699WHISXX94Z,5
8,B004A9SDD8,A2BXV49EIES2TB,5
9,B004A9SDD8,A37HM5TMCMHJES,5


In [238]:
dataframe.shape

(20000, 3)

In [239]:
# Create user-item matrix given a matrix and mapping from original matrix columns to new matrix rows/columns
def create_user_item_matrix(data, rowMapping, columnMapping):
    # Create table of users to items by taking row and column pivots
    # Rows of user-item table are users
    rows, row_pos = np.unique(data[:, rowMapping], return_inverse=True)
    # Columns of user-item table are movies
    cols, col_pos = np.unique(data[:, columnMapping], return_inverse=True)

    pivot_table = np.zeros((len(rows), len(cols)), dtype=data.dtype)
    pivot_table[row_pos, col_pos] = data[:, 2]
    return pivot_table, rows, cols

In [376]:
np.setdiff1d(test_df["reviewerID"].unique(),train_df["reviewerID"].unique()).shape

(2579,)

In [379]:
np.setdiff1d(test_df["asin"].unique(),train_df["asin"].unique()).shape

(0,)

In [241]:
"A08839553NG2E57YWLLVC" in dataframe["reviewerID"].unique().shape

False

In [243]:
dataframe["overall"].unique()

array([3, 5, 4, 1, 2])

In [244]:
ratings_matrix, rows, cols = create_user_item_matrix(values, rowMapping=1, columnMapping=0)
print (ratings_matrix.shape)

(15079, 255)


In [245]:
pd.DataFrame(ratings_matrix[:10], index = rows[:10], columns= cols)

,B004A9SDD8,B004AFQAUA,B004AHBBPW,B004ALVL6W,B004AMAIZQ,B004AMDC86,B004ANC00Q,B004ANE2WU,B004ANMWPY,B004AZH4C8,...,B004JYUSAA,B004JZ7ORY,B004JZBGWI,B004K4150G,B004K44R74,B004K46T52,B004K4M1AY,B004K4P2IC,B004K4QPXI,B004K4RY9M
A01961902L18E0CEAXEJU,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0220159ZRNBTRKLG08H,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A02574211ZTWG21TA1NB9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A046006938AM5SST8SS4D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A064321025DSGTER7QDIX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0698182HBSA2D2MEX40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
A07305593IWIWUBQUZ288,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A07347751OS3V85PQ4PGK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0759107CA9MPWVRF6VN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A08839553NG2E57YWLLVC,0,0,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [213]:
def get_index_map(arr):
    index_map = {}
    for el in arr:
        index_map[el] = len(index_map)
    return index_map

In [214]:
def pearson_coeff(target, others):
    return np.corrcoef(matrix[target], matrix[others])[0][1:]

In [337]:
# Get unique movies and users
unique_items = np.unique(values[:,0])
unique_users = np.unique(values[:,1])

In [338]:
unique_items.shape

(255,)

In [339]:
unique_users.shape

(15079,)

In [340]:
# Create mapping from unique list to 
item_idx_map = get_index_map(unique_items)
user_idx_map = get_index_map(unique_users)

In [252]:
mean_ratings = np.true_divide(ratings_matrix.sum(1), (ratings_matrix!=0).sum(1))

In [281]:
K = 0.1

In [282]:
def get_users_who_rated_item(item_id):
    return np.where(ratings_matrix[:,item_idx_map[item_id]] > 0)[0]

def get_users_who_rated_item_by_index(item_idx):
    return np.where(ratings_matrix[:,item_idx] > 0)

def predict(user_id, item_id):
    item_idx = item_idx_map[item_id]
    user_idx = user_idx_map[user_id]
    other_users_idx = get_users_who_rated_item(item_id)
    if user_idx in other_users_idx:
        np.delete(other_users_idx, user_idx)
    pearson_correlation = [pearsonr(other_user, ratings_matrix[user_idx])[0] for other_user in ratings_matrix[other_users_idx]]
    mean_others = mean_ratings[other_users_idx]
    rating_others = ratings_matrix[other_users_idx,item_idx]
    print "mean user = ", mean_ratings[user_idx]
    print "mean other = ", np.mean(mean_others)
    print "pearson = ", pearson_correlation
    prediction = mean_ratings[user_idx] + K * np.sum(pearson_correlation * (rating_others - mean_others))
    return prediction

In [284]:
predict(u'A0698182HBSA2D2MEX40', 'B004K44R74')

mean user =  3.66666666667
mean other =  3.77666666667
pearson =  [-0.019536071061484646, -0.006477409917721514, -0.010904761485729548, -0.007627724859868794, -0.009178526739238102, -0.006477409917721514, -0.009178526739238106, -0.0091785267392381, -0.006477409917721508, -0.006477409917721502]


3.6652073422360067

In [270]:
np.mean(mean_ratings)

4.050079148953411

In [298]:
test_dataframe, test_values = load_required_data('../reviews_Apps_for_Android_5.json', ["asin", "reviewerID", "overall"], 20001, 30000)

In [324]:
test_dataframe['reviewerID'].unique().shape

(8600,)

In [345]:
new_df = pd.DataFrame()
for index, row in test_dataframe.iterrows():
    if (row['reviewerID'] in unique_users and row['asin'] in unique_items):
        new_df = new_df.append(row)

In [ ]:
test_dataframe['reviewerID'].unique().shape

In [347]:
new_df.shape

(260, 3)

In [ ]:
def update_index_map(index_map, arr):
    for el in arr:
        index_map[el] = len(index_map)
    return index_map

In [299]:
# Returns DF containing predictions, MAE, RMSE
def get_predictions(dataframe):
    df = dataframe.copy()
    print ("Predicting ratings..")
    start_time = time.time()
    df['Prediction'] = df.apply(lambda row: predict(row['reviewerID'], row['asin']), axis=1)
    MAE = round(mean_absolute_error(df['overall'], df['Prediction']), 2)
    RMSE = round(np.sqrt(mean_squared_error(df['overall'], df['Prediction'])), 2)
    print "Done Predicting in",  round((time.time() - start_time), 2), "seconds"
    return df, MAE, RMSE

,asin,reviewerID,overall
4298,B004EBZX6W,AWO8OHKCI0Y0I,5
9201,B004GYY714,A10R9QFEFYAUZW,5
679,B004AZH4C8,A1VITR3L9Q6KY2,5
18780,B004JXUXS8,AYOBWKPV3H0V7,2
6955,B004GMJKTK,A1T6IOH2615C01,4
8571,B004GYCF7W,A2UH5YQZS3WXFY,5
9405,B004GYYQLA,A3G5MOHY1U635N,4
13699,B004HMJ5QM,A101Q7PGHKL3QF,5
3930,B004DPNBIQ,A1TEYEOQZIHTTZ,4
13573,B004HM35RC,A2888QFVFKC7PD,5
